# Extração de leis do município de Curitiba

Implementação de extração de leis do município de Curitiba apresentada ao Centro de Ciências Matemáticas Aplicadas à Indústria do Instituto de Ciências Matemáticas e de Computação, Universidade de São Paulo - ICMC/USP, como parte dos requisitos para obtenção do título de Especialista em Ciências de Dados.

**Área de concentração**: Ciências de Dados

**Orientador**: Prof. Dr. Luis Gustavo Nonato\
**Aluno**: Luís Henrique Paiva

## 1. Extração do texto dos arquivos em formato PDF

### 1.1 Instalação de bibliotecas de extração

In [1]:
%pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 77.7 MB/s eta 0:00:00


### 1.2 Importação das bibliotecas de extração

In [2]:
import os
import re
import pandas as pd
import numpy as np

from pdfminer.high_level import extract_text

### 1.3 Definição do diretório onde estão os arquivos PDFs

In [ ]:
# Para ambiente do Google Colab, alterar conforme o SO
diretorio = '/leis'

### 1.4 Definição das funções auxiliares para a extração do texto

In [4]:
def separa_colunas(texto):
    # Coluna 1: "numeracao" - Ocorrência da sequência "LEI Nº"
    numeracao = re.search(r'LEI Nº\s*(\d+)', texto)
    numeracao = numeracao.group(1) if numeracao else ""

    # Coluna 2: "sumula" - Texto entre o último dígito do número da lei e a ocorrência de "A CÂMARA MUNICIPAL DE CURITIBA"
    sumula = re.search(r"\d+\s*(.*?)A CÂMARA MUNICIPAL DE CURITIBA", texto, re.DOTALL)
    sumula = sumula.group(1).strip() if sumula else ""

    # Coluna 3: "corpo_lei" - Texto do primeiro ao último artigo antes do de publicação
    corpo_lei = re.search(r'(?<=seguinte lei:)(.*?)(PALÁCIO 29 DE MARÇO|Rafael Valdomiro Greca de Macedo)', texto, re.IGNORECASE)
    corpo_lei = corpo_lei.group(1).strip() if corpo_lei else ""

    return {
        "ano": "",
        "numeracao": numeracao,
        "sumula": sumula,
        "corpo_lei": corpo_lei
    }


def remove_espacos_extras(texto):
    # Substitui múltiplos espaços por um único espaço
    texto_limpo = re.sub(r'\s+', ' ', texto)

    # Remove espaços no início e no final do texto
    return texto_limpo.strip()


def extrair_texto_leis_pdf(diretorio):
    # Lista para armazenar os dados extraídos
    dados = []

    # Percorre todos os diretórios dentro da pasta
    for ano in os.listdir(diretorio):
        caminho_ano = os.path.join(diretorio, ano)

        # Verifica se é um diretório e se o nome do diretório é um ano válido
        if os.path.isdir(caminho_ano) and ano.isdigit():
            # Percorre todos os arquivos PDF no diretório de cada ano
            for filename in os.listdir(caminho_ano):
                if filename.endswith('.pdf'):
                    arquivo = os.path.join(caminho_ano, filename)

                    try:
                        # Extrai o texto do PDF
                        texto = extract_text(arquivo)

                        # Limpeza do texto: remove espaços extras
                        texto_limpo = remove_espacos_extras(texto)

                        # Separa o texto em colunas
                        dados_lei = separa_colunas(texto_limpo)

                        # Adiciona a coluna com ano de publicação da lei
                        dados_lei["ano"] = ano

                        # Adiciona os dados extraídos à lista
                        dados.append(dados_lei)

                    except Exception as e:
                        print(f"Erro ao processar o arquivo {filename}: {e}")

    # Cria o DataFrame com as leis extraídas
    df = pd.DataFrame(dados)

    return df

### 1.5 Extração e visualização do texto das leis

In [5]:
df_raw = extrair_texto_leis_pdf(diretorio)

In [6]:
df_raw.head(5)

,ano,numeracao,sumula,corpo_lei
0,2020,15797,Altera dispositivos da Lei Municipal nº 15.669...,"Art. 1º Ficam incluídas, alteradas e excluídas..."
1,2020,15798,Estima a Receita e fixa a Despesa do Município...,TÍTULO I DAS DISPOSIÇÕES PRELIMINARES Art. 1º ...
2,2020,15779,Autoriza o Poder Executivo a constituir s o c ...,Art. 1º Fica o Poder Executivo Municipal autor...
3,2020,15778,Dispõe sobre a geração de energia elétrica f o...,Art. 1º Os lagos de parques municipais poderão...
4,2020,15780,Cria o Pólo Gastronômico e Cultural do Petit B...,"Art. 1º Fica designado como ""Polo Gastronômico..."


In [7]:
df_raw.shape

(1527, 4)

### 1.6 Exportação do dataframe bruto (*raw*) contendo as leis para o formato CSV

In [8]:
df_raw.to_csv('raw.csv', sep='|', index=False, encoding='UTF-8')

### 1.7 Eliminação de leis duplicadas

In [9]:
def eliminar_duplicados(df):
    # Elimina as linhas duplicadas com base na coluna 'numeracao', mantendo a última ocorrência
    # (texto de lei mais atualizado)
    return df.drop_duplicates(subset=['numeracao'], keep='last')

In [10]:
df_interim = df_raw.copy()

In [11]:
df_interim = eliminar_duplicados(df_interim)

In [12]:
df_interim.head(5)

,ano,numeracao,sumula,corpo_lei
0,2020,15797,Altera dispositivos da Lei Municipal nº 15.669...,"Art. 1º Ficam incluídas, alteradas e excluídas..."
1,2020,15798,Estima a Receita e fixa a Despesa do Município...,TÍTULO I DAS DISPOSIÇÕES PRELIMINARES Art. 1º ...
2,2020,15779,Autoriza o Poder Executivo a constituir s o c ...,Art. 1º Fica o Poder Executivo Municipal autor...
3,2020,15778,Dispõe sobre a geração de energia elétrica f o...,Art. 1º Os lagos de parques municipais poderão...
4,2020,15780,Cria o Pólo Gastronômico e Cultural do Petit B...,"Art. 1º Fica designado como ""Polo Gastronômico..."


In [13]:
df_interim.shape

(1469, 4)

### 1.8 Exportação do dataframe intermediário (*interim*) contendo as leis para o formato CSV

In [14]:
df_interim.to_csv('interim.csv', sep='|', index=False, encoding='UTF-8')

### 1.9 Verificação de dados faltantes

In [15]:
df_preprocessed = df_interim.copy()

In [16]:
df_preprocessed.replace('', np.nan, inplace=True)

In [17]:
df_preprocessed = df_preprocessed.dropna()

In [18]:
df_preprocessed.isna().any().any()

np.False_

In [19]:
df_preprocessed.isna().sum()

,0
ano,0
numeracao,0
sumula,0
corpo_lei,0


### 1.10 Exportação do dataframe contendo as leis para o formato CSV

In [20]:
df_preprocessed.to_csv('preprocessed.csv', sep='|', index=False, encoding='UTF-8')